In [ ]:
# Import essential libraries
import torch.nn.functional as F
import networkx as nx
import matplotlib.pyplot as plt
from utils.graph_utils import *
import numpy as np

In [ ]:
# --- CREATE PATH 
import os
x = 1
!mkdir graphs
path= './graphs/' + str(x)
while os.path.exists(path):
  x+=1
  path= './graphs/' + str(x)
!mkdir {path}

In [ ]:
# --- Step 1: Create directed attack graph ---
G_original = nx.DiGraph()

G_original.add_nodes_from(["Data Server", "Pad", "Web Server", "Host 1", "Host 2", "Host 3", "File Server",
                              "Router" # ADDED: New central node
                           ])

# --- Step 2: Update node attributes (priority + state) ---
node_attributes = {
    "Data Server": {"state": 0, "priority": 2},
    "Pad": {"state": 0, "priority": 1},
    "Web Server": {"state": 0, "priority": 1},
    "Host 1": {"state": 0, "priority": 1},
    "Host 2": {"state": 0, "priority": 0},
    "Host 3": {"state": 0, "priority": 0},
    "File Server": {"state": 0, "priority": 0},
    "Router": {"state": 0, "priority": 2} # ADDED: High priority for a central networking device
}
nx.set_node_attributes(G_original, node_attributes)
# --- Step 3: Update edges with weights (user/root access probabilities) ---
edges = [
    # Original Edges (Kept for continuity)
    ("Pad", "Host 1", {"user": 0.6, "root": 0.48}),
    ("Pad", "Host 2", {"user": 0.32, "root": 0.32}),
    ("Pad", "Host 3", {"user": 0.32, "root": 0.32}),
    ("Pad", "Web Server", {"user": 0.8, "root": 0.6}),
    ("Host 1", "Pad", {"user": 0.6, "root": 0.6}),
    ("Host 1", "Web Server", {"user": 0.8, "root": 0.6}),
    ("Host 1", "Host 2", {"user": 0.32, "root": 0.32}),
    ("Host 1", "Host 3", {"user": 0.32, "root": 0.32}),
    ("Host 2", "Host 3", {"user": 0.8, "root": 0.8}),
    ("Host 2", "File Server", {"user": 0.8, "root": 0.6}),
    ("Host 2", "Data Server", {"user": 0.8, "root": 0.6}),
    ("Host 3", "Host 2", {"user": 0.8, "root": 0.8}),
    ("Host 3", "File Server", {"user": 0.8, "root": 0.6}),
    ("Host 3", "Data Server", {"user": 0.8, "root": 0.6}),
    ("Web Server", "File Server", {"user": 0.8, "root": 0.04}),
    ("Web Server", "Data Server", {"user": 0.8, "root": 0.04}),
    ("File Server", "Data Server", {"user": 0.8, "root": 0.04}),

    # NEW EDGES (To improve realism)

    # Data Server to File Server (Back-path for logging/backup)
    ("Data Server", "File Server", {"user": 0.6, "root": 0.02}),

    # Core Server to Router and vice-versa (All major traffic goes through the router)
    ("Router", "Web Server", {"user": 0.9, "root": 0.9}), # High access probability
    ("Web Server", "Router", {"user": 0.9, "root": 0.9}),
    ("Router", "Data Server", {"user": 0.9, "root": 0.1}), # High user access, but very restricted root access to data server
    ("Data Server", "Router", {"user": 0.9, "root": 0.9}),

    # Pad/Workstation to Router (External access point)
    ("Pad", "Router", {"user": 0.7, "root": 0.5}),
]


G_original.add_edges_from(edges)
node_order = list(G_original.nodes())
node_features = torch.tensor([[G_original.nodes[n]['state'], G_original.nodes[n]['priority']] for n in node_order], dtype=torch.float32)
edge_features = torch.tensor([[d['user'], d['root']] for u, v, d in G_original.edges(data=True)], dtype=torch.float32)
g_dgl = dgl.from_networkx(G_original)
g_dgl.ndata['h'] = node_features
g_dgl.edata['h'] = edge_features

# --- Visualize the graph ---
plt.figure(figsize=(8, 6))
pos = nx.spring_layout(G_original)
nx.draw(G_original, pos, with_labels=True, node_color='lightblue', node_size=500, font_size=10)
edge_labels = nx.get_edge_attributes(G_original, 'user')
nx.draw_networkx_edge_labels(G_original, pos, edge_labels=edge_labels)
plt.title("Attack Graph (Edge Weights: User Access Prob)")
plt.show()

In [ ]:
# --- 1. Chuyển đổi đồ thị sang DGL và trích xuất đặc trưng ---

# DGL yêu cầu các nút phải có ID là số nguyên. Tạo một danh sách các nút theo thứ tự.
node_order = list(G_original.nodes())

# Trích xuất đặc trưng của các nút theo đúng thứ tự và chuyển thành Tensor.
# 2 đặc trưng: 'state' và 'priority'.
node_features_list = [G_original.nodes[n] for n in node_order]
node_features = torch.tensor([[n['state'], n['priority']] for n in node_features_list], dtype=torch.float32)

# Trích xuất đặc trưng của các cạnh và chuyển thành Tensor.
# 2 đặc trưng: 'user' và 'root'.
edge_features_list = [d for u, v, d in G_original.edges(data=True)]
edge_features = torch.tensor([[e['user'], e['root']] for e in edge_features_list], dtype=torch.float32)

# Chuyển đổi đồ thị networkx sang DGL.
# node_attrs và edge_attrs chỉ định tên các thuộc tính cần sao chép.
g_dgl = dgl.from_networkx(G_original, node_attrs=['state', 'priority'], edge_attrs=['user', 'root'])

# Gán các Tensor đặc trưng đã tạo vào đồ thị DGL.
# Dùng tên 'h' để thống nhất với class đã viết.
g_dgl.ndata['h'] = node_features
g_dgl.edata['h'] = edge_features

print("--- Thông tin đồ thị DGL ---")
print(g_dgl)
print("\nĐặc trưng các nút (Node Features) shape:", g_dgl.ndata['h'].shape)
print("Đặc trưng các cạnh (Edge Features) shape:", g_dgl.edata['h'].shape)


In [ ]:
# --- Training loop (CELL: a797878cc131e7cc) ---
# Đảm bảo bạn đã import các class từ file 'tgn_dgi_encoder.py'
# from tgn_dgi_encoder import TGNWrapperEncoder, DGI
# (Hoặc chạy cell chứa định nghĩa class ở trên)

NDIM_IN = g_dgl.ndata['h'].shape[1]  # 2 (state, priority)
EDIM = g_dgl.edata['h'].shape[1]    # 2 (user, root)
N_HIDDEN = 16 # Kích thước ẩn cho EGraphSAGE
N_OUT = 24    # Kích thước đầu ra cho EGraphSAGE
N_LAYERS = 2
EPOCHS = 2000
LEARNING_RATE = 0.001

# --- THAY ĐỔI Ở ĐÂY ---

# 1. Định nghĩa các tham số cho TGN
MEMORY_DIM = 32 # Kích thước bộ nhớ TGN
MSG_DIM = 32    # Kích thước message TGN
NUM_NODES = g_dgl.num_nodes() # Số nút ban đầu

# 2. Khởi tạo TGNWrapperEncoder
# Wrapper này sẽ tự khởi tạo EGraphSAGE bên trong nó
encoder = TGNWrapperEncoder(
    num_nodes=NUM_NODES,
    node_feat_dim=NDIM_IN,
    edge_feat_dim=EDIM,
    memory_dim=MEMORY_DIM,
    msg_dim=MSG_DIM,
    sage_n_hidden=N_HIDDEN,
    sage_n_out=N_OUT,
    sage_n_layers=N_LAYERS,
    sage_activation=F.leaky_relu
)

# 3. Khởi tạo DGI (với TGNWrapperEncoder)
dgi_model = DGI(encoder)

# --- HẾT THAY ĐỔI ---

# Khởi tạo optimizer
optimizer = torch.optim.Adam(dgi_model.parameters(), lr=LEARNING_RATE)

print("\n--- Bắt đầu quá trình huấn luyện DGI (với TGN Wrapper) ---")
for epoch in range(EPOCHS):
    dgi_model.train()
    optimizer.zero_grad()
    
    # Hàm forward của TGNWrapperEncoder -> EGraphSAGE -> DGI sẽ được gọi
    loss = dgi_model(g_dgl, g_dgl.ndata['h'], g_dgl.edata['h'])

    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{EPOCHS}], Loss: {loss.item():.4f}')

print("--- Huấn luyện hoàn tất! ---")

# --- CẬP NHẬT CELL [ d7bfb809f914600b ] ---
# Khi lấy embedding cuối cùng, hãy đảm bảo bạn gọi đúng:
dgi_model.eval() 
with torch.no_grad():
    # Gọi DGI.encoder (chính là TGNWrapperEncoder)
    # Nó sẽ trả về (node_emb, edge_emb)
    final_node_embeddings, final_edge_embeddings = dgi_model.encoder(
        g_dgl, g_dgl.ndata['h'], g_dgl.edata['h'], corrupt=False
    )


In [ ]:
import pandas as pd
import seaborn as sns
# --- Lấy graphs cuối cùng từ bộ mã hóa đã được huấn luyện ---
dgi_model.eval() # Chuyển mô hình sang chế độ đánh giá
with torch.no_grad(): # Không tính toán gradient nữa
    final_node_embeddings, final_edge_embeddings = dgi_model.encoder(g_dgl, g_dgl.ndata['h'], g_dgl.edata['h'])

# Chuyển tensor sang numpy để xử lý
nodes_emb_np = final_node_embeddings.detach().numpy()
edges_emb_np = final_edge_embeddings.detach().numpy()



# 1. Tạo Figure và Axes
fig, ax = plt.subplots(figsize=(8, 6))

# 2. Vẽ biểu đồ lên Axes (ax)
pos = nx.spring_layout(G_original)
nx.draw(G_original, pos, ax=ax, with_labels=True, node_color='lightblue', node_size=500, font_size=10)
edge_labels = nx.get_edge_attributes(G_original, 'user')
nx.draw_networkx_edge_labels(G_original, pos, ax=ax, edge_labels=edge_labels)
ax.set_title("Attack Graph (Edge Weights: User Access Prob)")

# 3. Bây giờ, biến 'fig' chính là thứ bạn muốn lưu
# Bạn có thể lưu nó ngay lập tức:
fig.savefig(path + "/attack_graph.png")




# --- Hiển thị dưới dạng bảng (Pandas) ---

print("\n--- Bảng Embedding cho các Nút (sau khi huấn luyện) ---")
# Tạo tên cho các cột embedding (ví dụ: emb_0, emb_1, ...)
node_emb_cols = [f'emb_{i}' for i in range(nodes_emb_np.shape[1])]
# Tạo DataFrame
df_nodes = pd.DataFrame(nodes_emb_np, index=node_order, columns=node_emb_cols)
pd.set_option('display.max_columns', None)
print(df_nodes)


print("\n--- Bảng Embedding cho các Cạnh (sau khi huấn luyện) ---")
# Tạo một DataFrame mới cho các cạnh với thông tin nút nguồn và nút đích
original_edges_list = list(G_original.edges())
edge_data = {
    'Source': [u for u, v in original_edges_list],
    'Destination': [v for u, v in original_edges_list]
}


In [ ]:
edge_emb_cols = [f'emb_{i}' for i in range(edges_emb_np.shape[1])]
df_edges = pd.DataFrame(edge_data)
df_edges_emb = pd.DataFrame(edges_emb_np, columns=edge_emb_cols)
df_edges_final = pd.concat([df_edges, df_edges_emb], axis=1)
print(df_edges_final)


# --- Trực quan hóa bằng Heatmap (Seaborn) ---
try:
    plt.rcParams['font.family'] = 'sans-serif'

    # Heatmap cho Node Embeddings
    plt.figure(figsize=(10, 6)) # Tùy chỉnh kích thước hình
    sns.heatmap(df_nodes, annot=True, cmap='winter', fmt='.2f')
    plt.title('Heatmap của Node Embeddings', fontsize=16)
    plt.xlabel('Dimension', fontsize=12)
    plt.ylabel('Nodes', fontsize=12)
    plt.tight_layout()
    print("\n[THÀNH CÔNG] Đã lưu heatmap của node graphs vào file 'node_embeddings_heatmap.png'")

    # Heatmap cho Edge Embeddings
    # Tạo nhãn cho trục y của heatmap cạnh
    edge_labels = [f'{u} -> {v}' for u, v in original_edges_list]
    plt.figure(figsize=(12, 10)) # Tùy chỉnh kích thước hình
    sns.heatmap(df_edges_emb, yticklabels=edge_labels, annot=True, cmap='winter', fmt='.2f')
    plt.title('Heatmap của Edge Embeddings', fontsize=16)
    plt.xlabel('Dimension', fontsize=12)
    plt.ylabel('Edges', fontsize=12)
    plt.tight_layout()
    print("[THÀNH CÔNG] Đã lưu heatmap của edge graphs vào file 'edge_embeddings_heatmap.png'")

except Exception as e:
    print(f"\n[LỖI] Không thể tạo heatmap. Vui lòng đảm bảo bạn đã cài đặt matplotlib và seaborn.")
    print(f"Lỗi chi tiết: {e}")

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.manifold import TSNE

# Chuyển graphs cạnh sang numpy array
edge_embeddings_np = final_edge_embeddings.detach().numpy()
edge_labels = [f'{u} -> {v}' for u, v in G_original.edges()]

# Sử dụng t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=min(5, len(edge_labels)-1))
embeddings_2d = tsne.fit_transform(edge_embeddings_np)

# Vẽ biểu đồ
plt.figure(figsize=(12, 10))
plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1])

# Thêm nhãn
for i, label in enumerate(edge_labels):
    plt.annotate(label, (embeddings_2d[i, 0], embeddings_2d[i, 1]))

plt.title('Trực quan hóa Edge Embeddings bằng t-SNE')
plt.show()

In [ ]:
# Lấy node graphs và bản đồ node
node_embeddings_np = final_node_embeddings.detach().numpy()
node_map = {name: i for i, name in enumerate(node_order)}

# Tạo lại danh sách các cạnh không tồn tại (mẫu âm)
np.random.seed(42) # Đặt seed để kết quả có thể tái tạo
positive_edges = list(G_original.edges())
num_positive = len(positive_edges)
negative_edges = []
all_nodes = list(G_original.nodes())
while len(negative_edges) < num_positive:
    u, v = np.random.choice(all_nodes, 2, replace=False)
    if not G_original.has_edge(u, v) and (u, v) not in negative_edges:
        negative_edges.append((u, v))

# Tạo đặc trưng cho các cạnh không tồn tại
negative_edge_features = []
for u, v in negative_edges:
    u_emb = node_embeddings_np[node_map[u]]
    v_emb = node_embeddings_np[node_map[v]]
    edge_feature = u_emb * v_emb # Phép nhân Hadamard
    negative_edge_features.append(edge_feature)
X_negative = np.array(negative_edge_features)

# Hiển thị dưới dạng bảng
print("\n--- Bảng Embedding cho các Cạnh KHÔNG Tồn Tại (Mẫu Âm) ---")
edge_labels = [f'{u} -> {v}' for u, v in negative_edges]
edge_emb_cols = [f'emb_{i}' for i in range(X_negative.shape[1])]
df_negative_edges = pd.DataFrame(X_negative, index=edge_labels, columns=edge_emb_cols)
pd.set_option('display.max_columns', None) # Đảm bảo in ra tất cả các cột
print(df_negative_edges)

In [ ]:
# Chuẩn bị dữ liệu embedding cho cả hai loại cạnh ---

# Lấy embedding của các cạnh có thật (mẫu dương)
positive_embeddings_np = final_edge_embeddings.detach().numpy()
positive_edges = list(G_original.edges())
positive_labels = [f'{u} -> {v}' for u, v in positive_edges]
num_positive = len(positive_edges)

# Tạo và tính embedding cho các cạnh không tồn tại (mẫu âm)
np.random.seed(42) # Đặt seed để kết quả có thể tái tạo
node_embeddings_np = final_node_embeddings.detach().numpy()
node_map = {name: i for i, name in enumerate(node_order)}

negative_edges = []
all_nodes = list(G_original.nodes())
while len(negative_edges) < num_positive:
    u, v = np.random.choice(all_nodes, 2, replace=False)
    if not G_original.has_edge(u, v) and (u, v) not in negative_edges:
        negative_edges.append((u, v))

negative_edge_features = []
for u, v in negative_edges:
    u_emb = node_embeddings_np[node_map[u]]
    v_emb = node_embeddings_np[node_map[v]]
    edge_feature = u_emb * v_emb
    negative_edge_features.append(edge_feature)
negative_embeddings_np = np.array(negative_edge_features)
negative_labels = [f'{u} -> {v} (âm)' for u, v in negative_edges]


# Kết hợp tất cả embedding và chạy t-SNE ---

# Kết hợp embedding dương và âm
all_embeddings_np = np.vstack([positive_embeddings_np, negative_embeddings_np])
all_labels = positive_labels + negative_labels

# Chạy t-SNE trên toàn bộ dữ liệu
tsne = TSNE(n_components=2, random_state=42, perplexity=min(10, len(all_labels)-1))
embeddings_2d = tsne.fit_transform(all_embeddings_np)


# Vẽ biểu đồ với màu sắc khác nhau ---

plt.figure(figsize=(14, 12))

# Vẽ các điểm của cạnh có thật (màu xanh)
plt.scatter(embeddings_2d[:num_positive, 0], embeddings_2d[:num_positive, 1],
            c='skyblue', label='Cạnh Tồn Tại (Dương)', s=100)

# Vẽ các điểm của cạnh không tồn tại (màu đỏ)
plt.scatter(embeddings_2d[num_positive:, 0], embeddings_2d[num_positive:, 1],
            c='salmon', label='Cạnh Không Tồn Tại (Âm)', s=50, alpha=0.7)

# Thêm nhãn cho từng điểm
for i, label in enumerate(all_labels):
    plt.annotate(label, (embeddings_2d[i, 0], embeddings_2d[i, 1]), fontsize=9)

plt.title('Trực quan hóa Edge Embeddings (Dương và Âm) bằng t-SNE')
plt.xlabel('Thành phần t-SNE 1')
plt.ylabel('Thành phần t-SNE 2')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
print("\n--- Trực quan hóa Node Embeddings bằng t-SNE (theo Priority) ---")

# Chuyển graphs sang numpy array
embeddings_np = final_node_embeddings.detach().numpy()

# Sử dụng t-SNE để giảm chiều dữ liệu xuống 2 chiều
tsne = TSNE(n_components=2, random_state=42, perplexity=min(5, len(node_order)-1))
embeddings_2d = tsne.fit_transform(embeddings_np)

# Tạo DataFrame để dễ dàng vẽ biểu đồ
df = pd.DataFrame(embeddings_2d, columns=['x', 'y'])
df['node_name'] = node_order
df['priority'] = [node_attributes[name]['priority'] for name in node_order]

# Vẽ biểu đồ
plt.figure(figsize=(12, 10))
sns.scatterplot(
    data=df,
    x='x',
    y='y',
    hue='priority',
    palette=sns.color_palette("hls", n_colors=df['priority'].nunique()),
    s=150, # Kích thước điểm
    alpha=0.8
)

# Thêm nhãn cho từng điểm
for i, row in df.iterrows():
    plt.text(row.x + 0.1, row.y + 0.1, row.node_name, fontsize=10)

plt.title('Trực quan hóa Node Embeddings bằng t-SNE (Tô màu theo Priority)', fontsize=16)
plt.xlabel('Thành phần t-SNE 1', fontsize=12)
plt.ylabel('Thành phần t-SNE 2', fontsize=12)
plt.legend(title='Priority')
plt.grid(True)
plt.show()


In [ ]:
from scipy.spatial.distance import pdist, squareform

print("\n--- Biểu diễn Ma trận Khoảng cách dưới dạng Heatmap ---")
plt.rcParams['text.color'] = 'black'
# Chuyển graphs sang numpy array
embeddings_np = final_node_embeddings.detach().numpy()

# Tính toán ma trận khoảng cách
distance_condensed = pdist(embeddings_np, metric='euclidean')
distance_matrix = squareform(distance_condensed)

# Tạo DataFrame để có nhãn cho biểu đồ
df_distance = pd.DataFrame(distance_matrix, index=node_order, columns=node_order)

# Vẽ heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(
    df_distance,
    annot=True,        # Hiển thị giá trị số trên ô
    annot_kws={"color": "white"},
    cmap="winter",  # Chọn bảng màu (viridis_r: màu đậm cho giá trị thấp)
    fmt=".2f"          # Định dạng số hiển thị (2 chữ số sau dấu phẩy)
)

plt.title('Heatmap Ma trận Khoảng cách Euclid giữa các Node', fontsize=16)
plt.tight_layout() # Tự động điều chỉnh cho vừa vặn
print("\n[THÀNH CÔNG] Đã lưu heatmap của ma trận khoảng cách vào file 'node_distance_heatmap.png'")
plt.show()

In [ ]:
# --- EXPORT EMBEDDINGS ---

np.save(f'graphs/{x}/node_embeddings.npy', nodes_emb_np)
np.save(f'graphs/{x}/edge_embeddings.npy', edges_emb_np)
print("\n[THÀNH CÔNG] Đã lưu graphs vào 'node_embeddings.npy' và 'edge_embeddings.npy'")


In [ ]:
# --- EXPORT GRAPH ---
import torch

# --- Lấy các biến tĩnh từ các cell trước ---
# G_original (từ cell 2)
# node_order (từ cell 3)
# node_features (features gốc, từ cell 3)
# edge_features (features gốc, từ cell 3)

# --- Tạo dictionary để lưu ---
static_data_to_save = {
    'G_original': G_original,
    'node_order': node_order,
    'node_map': {name: i for i, name in enumerate(node_order)},

    # Features gốc (đầu vào của GNN)
    'node_features_original': node_features,
    'edge_features_original': edge_features,
}

# --- Lưu vào một file duy nhất ---
STATIC_FILE_PATH = path + "/graph_environment.pth"
torch.save(static_data_to_save, STATIC_FILE_PATH)

print(f"[THÀNH CÔNG] Đã lưu dữ liệu môi trường tĩnh vào file: {STATIC_FILE_PATH}")

In [ ]:
# --- LƯU TRỌNG SỐ CỦA MÔ HÌNH (STATE DICTIONARY) ---
MODEL_STATE_PATH = path + "/dgi_model_state_dict.pth"

# Lưu state_dict của mô hình DGI
torch.save(dgi_model.state_dict(), MODEL_STATE_PATH)

print(f"\n[THÀNH CÔNG] Đã lưu trọng số của mô hình vào file '{MODEL_STATE_PATH}'")

In [ ]:
# --- EXPORT MODEL PARAMETERS ---

import torch

# --- Định nghĩa các tham số GNN ---
NDIM_IN = g_dgl.ndata['h'].shape[1]
EDIM = g_dgl.edata['h'].shape[1]
N_HIDDEN = 16
N_OUT = 24
N_LAYERS = 2
# (Các tham số khác như EPOCHS, LEARNING_RATE không cần thiết cho
# việc *khởi tạo* lại mô hình, nhưng bạn có thể thêm vào nếu muốn)

# --- Tạo dictionary chứa các tham số cần thiết cho việc tải mô hình ---
model_config = {
    'NDIM_IN': NDIM_IN,
    'EDIM': EDIM,
    'N_HIDDEN': N_HIDDEN,
    'N_OUT': N_OUT,
    'N_LAYERS': N_LAYERS
}

# --- Lưu dictionary vào file ---
CONFIG_FILE_PATH = path + "/model_config.pth"
torch.save(model_config, CONFIG_FILE_PATH)

print(f"[THÀNH CÔNG] Đã lưu cấu hình mô hình vào file: {CONFIG_FILE_PATH}")
print("Cấu hình đã lưu:", model_config)

In [ ]:
# # --- ORIGINAL GRAPH ---
# # --- Step 1: Create directed attack graph ---
# G_original = nx.DiGraph()
# G_original.add_nodes_from(["Data Server", "Pad", "Web Server", "Host 1", "Host 2", "Host 3", "File Server"
#                               # , "New Node"
#                            ])
#
# # --- Step 2: Add node attributes (priority + state) ---
# node_attributes = {
#     "Data Server": {"state": 0, "priority": 2},
#     "Pad": {"state": 0, "priority": 1},
#     "Web Server": {"state": 0, "priority": 1},
#     "Host 1": {"state": 0, "priority": 1},
#     "Host 2": {"state": 0, "priority": 0},
#     "Host 3": {"state": 0, "priority": 0},
#     "File Server": {"state": 0, "priority": 0},
#     # "New Node": {"state": 0, "priority": 0}
# }
# nx.set_node_attributes(G_original, node_attributes)
#
# # --- Step 3: Add edges with weights (user/root access probabilities) ---
# edges = [
#     ("Pad", "Host 1", {"user": 0.6, "root": 0.48}),
#     ("Pad", "Host 2", {"user": 0.32, "root": 0.32}),
#     ("Pad", "Host 3", {"user": 0.32, "root": 0.32}),
#     ("Pad", "Web Server", {"user": 0.8, "root": 0.6}),
#     ("Host 1", "Pad", {"user": 0.6, "root": 0.6}),
#     ("Host 1", "Web Server", {"user": 0.8, "root": 0.6}),
#     ("Host 1", "Host 2", {"user": 0.32, "root": 0.32}),
#     ("Host 1", "Host 3", {"user": 0.32, "root": 0.32}),
#     ("Host 2", "Host 3", {"user": 0.8, "root": 0.8}),
#     ("Host 2", "File Server", {"user": 0.8, "root": 0.6}),
#     ("Host 2", "Data Server", {"user": 0.8, "root": 0.6}),
#     ("Host 3", "Host 2", {"user": 0.8, "root": 0.8}),
#     ("Host 3", "File Server", {"user": 0.8, "root": 0.6}),
#     ("Host 3", "Data Server", {"user": 0.8, "root": 0.6}),
#     ("Web Server", "File Server", {"user": 0.8, "root": 0.04}),
#     ("Web Server", "Data Server", {"user": 0.8, "root": 0.04}),
#     ("File Server", "Data Server", {"user": 0.8, "root": 0.04}),
#     # ("Data Server", "New Node", {"user": 0.8, "root": 0.04})
# ]